In [ ]:
#import ee
#ee.Authenticate()

In [376]:
import ee
import geemap

import pandas as pd
import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import MultiPolygon, Polygon
from shapely import wkb

from amigocloud import AmigoCloud

In [377]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [378]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [379]:
# VARIEBLES GLOBALES

GDF_LOTES_CLOUD = None
FC_LOTES = None

In [380]:
# FUNCIONES

# funcion para convertir de formato wkb a geometria
def convertir_wkb(wkb_data):
    return wkb.loads(wkb_data, hex=True)[0]

# Función para convertir un polígono en un multipolígono
def poligono_a_multipoligono(poligono):
    return MultiPolygon([poligono])

In [381]:
# PARAMETROS DE VISUZALIZACION

# parametro para ver lostes de credito ya registrados
vis_lotes = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [382]:
def cargar_lotes_cloud():
    global FC_LOTES, GDF_LOTES_CLOUD
    # seleccionar todos los lotes de credito siembra con un cursos (son mas de 100 elementos)
    query = {'query': 'select * from dataset_307563'}
    # cursor
    select = amigocloud.get_cursor('https://app.amigocloud.com/api/v1/projects/32672/sql', query)

    # crear un dataframe vacio
    df_lotes = pd.DataFrame()
    # recorrer el cursor (.next), e ir agregando los lotes al dataframe
    for i in select:
        fila = pd.Series(i)
        df_lotes = df_lotes.append(fila, ignore_index=True)
    
    # del dataframe, seleccionar solo los datos amigo_id, nombre del lote, y geometria
    df_lotes = df_lotes[['amigo_id', 'lote', 'geometria']]

    # convertir el campo geometria de wkb a geometrico
    df_lotes['geometria'] = df_lotes['geometria'].apply(convertir_wkb)

    # convertir DataFrame a GeoDataFrame
    GDF_LOTES_CLOUD = gpd.GeoDataFrame(df_lotes, geometry='geometria')

In [383]:
def cargar_gdf_lotes_map():
    # Convierte el GeoDataFrame en una lista de características
    features = GDF_LOTES_CLOUD.__geo_interface__["features"]
    # Convierte la lista de características en una FeatureCollection de Google Earth Engine y guargar en variable global
    FC_LOTES = ee.FeatureCollection(features)
    Map.addLayer(FC_LOTES.style(**vis_lotes), {}, 'Lotes')

In [384]:
cargar_lotes_cloud()

In [386]:
cargar_gdf_lotes_map()

In [385]:
Map = geemap.Map(basemap="SATELLITE")

In [387]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [388]:
# capturar los ROIs
lotes_roi = Map.user_rois
fc_roi = ee.FeatureCollection(lotes_roi)

# Convierte la FeatureCollection en un diccionario
fc_dict = fc_roi.getInfo()
# Extrae las características del diccionario
features_list = fc_dict['features']
# Convierte las características en un GeoDataFrame
gdf_lotes = gpd.GeoDataFrame.from_features(features_list)

# convertir poligonos a multipoligonos
gdf_lotes['geometry'] = gdf_lotes['geometry'].apply(poligono_a_multipoligono)

# extrae el poligono de la ultima posicion
poligono = gdf_lotes.iloc[len(gdf_lotes) - 1]['geometry']

# insert en amigocloud
query = {'query': 'insert into dataset_307563 (geometria) values (ST_GeomFromText(\'' + str(poligono) + '\',4326))'}
res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)

# eliminar los ROIs
while Map.draw_features:
    Map.remove_last_drawn()

# agrega la nueva geometria al GDF_LOTES_CLOUD
fila = {'amigo_id':'', 'lote':'', 'geometria':poligono}
GDF_LOTES_CLOUD = GDF_LOTES_CLOUD.append(fila, ignore_index=True)

cargar_gdf_lotes_map()

# recorrer los gdf_lotes y cargar los poligonos a amigocloud
'''
for i in range(len(gdf_lotes)):
    poligono = gdf_lotes.iloc[i]['geometry']
    query = {'query': 'insert into dataset_307563 (geometria) values (ST_GeomFromText(\'' + str(poligono) + '\',4326))'}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    #response = amigocloud.sql('32672', query)
    print(i)
'''

"\nfor i in range(len(gdf_lotes)):\n    poligono = gdf_lotes.iloc[i]['geometry']\n    query = {'query': 'insert into dataset_307563 (geometria) values (ST_GeomFromText('' + str(poligono) + '',4326))'}\n    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)\n    #response = amigocloud.sql('32672', query)\n    print(i)\n"

In [331]:
print(poligono)

MULTIPOLYGON (((-63.222198 -17.287947, -63.218701 -17.287025, -63.216791 -17.291901, -63.221898 -17.292045, -63.222198 -17.287947)))


In [332]:
gdf_lotes

,geometry
0,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."


In [333]:
GDF_LOTES_CLOUD

,amigo_id,lote,geometria
0,c25021f58f7848bdbf6c5fdee28af591,L9,"POLYGON ((-63.10632 -17.19889, -63.10765 -17.1..."
1,8880a15eb53a40dbb5ade5cefa56722f,L1.3,"POLYGON ((-63.11747 -17.20613, -63.11691 -17.2..."
2,2ff903533d474d20a64ab54303eb0437,L1,"POLYGON ((-62.75349 -17.24376, -62.75357 -17.2..."
3,b33bd293439a4d638c7d95700b9b0bac,L14.3,"POLYGON ((-63.01162 -17.33073, -63.01157 -17.3..."
4,f538ccc8293e4ae699d8167d78283130,L7.5,"POLYGON ((-63.03620 -17.02499, -63.03587 -17.0..."
...,...,...,...
480,54b87f3ca9ab4fca8b650e4de1539447,L12.1,"POLYGON ((-62.95645 -17.13284, -62.95583 -17.1..."
481,a3c87de6be714afaa5e7113144dcc92a,L13.2,"POLYGON ((-62.95229 -17.13327, -62.95395 -17.1..."
482,caa64d2a4c22423f9dbd3286e23c06eb,L14.1,"POLYGON ((-63.22369 -17.39424, -63.22833 -17.3..."
483,1ae18a6ff5274250a00c70e96f2188a7,L7,"POLYGON ((-63.16044 -17.31626, -63.15031 -17.3..."


In [337]:
fila = {'amigo_id':'', 'lote':'', 'geometria':poligono}

In [340]:
GDF_LOTES_CLOUD = GDF_LOTES_CLOUD.append(fila, ignore_index=True)

In [339]:
GDF_LOTES_CLOUD

,amigo_id,lote,geometria
0,c25021f58f7848bdbf6c5fdee28af591,L9,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
1,8880a15eb53a40dbb5ade5cefa56722f,L1.3,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
2,2ff903533d474d20a64ab54303eb0437,L1,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
3,b33bd293439a4d638c7d95700b9b0bac,L14.3,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
4,f538ccc8293e4ae699d8167d78283130,L7.5,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
...,...,...,...
480,54b87f3ca9ab4fca8b650e4de1539447,L12.1,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
481,a3c87de6be714afaa5e7113144dcc92a,L13.2,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
482,caa64d2a4c22423f9dbd3286e23c06eb,L14.1,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."
483,1ae18a6ff5274250a00c70e96f2188a7,L7,"MULTIPOLYGON (((-63.22220 -17.28795, -63.21870..."


In [341]:
GDF_LOTES_CLOUD

,amigo_id,lote,geometria
0,c25021f58f7848bdbf6c5fdee28af591,L9,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
1,8880a15eb53a40dbb5ade5cefa56722f,L1.3,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
2,2ff903533d474d20a64ab54303eb0437,L1,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
3,b33bd293439a4d638c7d95700b9b0bac,L14.3,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
4,f538ccc8293e4ae699d8167d78283130,L7.5,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
...,...,...,...
481,a3c87de6be714afaa5e7113144dcc92a,L13.2,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
482,caa64d2a4c22423f9dbd3286e23c06eb,L14.1,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
483,1ae18a6ff5274250a00c70e96f2188a7,L7,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
484,04b1f4397eaa44eb932ba9964df4a6f2,L5,"(POLYGON ((-63.222198 -17.287947, -63.218701 -..."
